In [5]:
import pandas as pd
import pyodbc

In [6]:
print("🔍 Drivers disponibles:")
for driver in pyodbc.drivers():
    print(f" - {driver}")

🔍 Drivers disponibles:
 - SQL Server
 - SQL Server Native Client RDA 11.0
 - ODBC Driver 17 for SQL Server
 - Microsoft Access Driver (*.mdb, *.accdb)
 - Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)
 - Microsoft Access Text Driver (*.txt, *.csv)
 - Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)
 - ODBC Driver 18 for SQL Server


In [7]:
import pyodbc


# Set up your connection string
server = 'class3-server.database.windows.net'
database = 'class3-database'
username = 'abraham'
password = 'Server123'

# Define the connection string for SQL Server (Azure)
conn_str = (
    "DRIVER={ODBC Driver 18 for SQL Server};"
    f'SERVER={server};'
    f'DATABASE={database};'
    f'UID={username};'
    f'PWD={password}'
    
)

# Establish the connection
try:
    conn = pyodbc.connect(conn_str)
    print("Connected to the Azure SQL Database successfully!")

except pyodbc.Error as e:
    print(f"Error connecting to the database: {e}")


query = "SELECT * FROM SalesLT.customer"

Connected to the Azure SQL Database successfully!


In [ ]:
datos = pd.read_sql(query, conn)
datos.head()

c:\Users\abrah\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,CustomerID,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,CompanyName,SalesPerson,EmailAddress,Phone,PasswordHash,PasswordSalt,rowguid,ModifiedDate
0,1,False,Mr.,Orlando,N.,Gee,None,A Bike Store,adventure-works\pamela0,orlando0@adventure-works.com,245-555-0173,L/Rlwxzp4w7RWmEgXX+/A7cXaePEPcp+KwQhl2fJL7w=,1KjXYs4=,3F5AE95E-B87D-4AED-95B4-C3797AFCB74F,2005-08-01
1,2,False,Mr.,Keith,None,Harris,None,Progressive Sports,adventure-works\david8,keith0@adventure-works.com,170-555-0127,YPdtRdvqeAhj6wyxEsFdshBDNXxkCXn+CRgbvJItknw=,fs1ZGhY=,E552F657-A9AF-4A7D-A645-C429D6E02491,2006-08-01
2,3,False,Ms.,Donna,F.,Carreras,None,Advanced Bike Components,adventure-works\jillian0,donna0@adventure-works.com,279-555-0130,LNoK27abGQo48gGue3EBV/UrlYSToV0/s87dCRV7uJk=,YTNH5Rw=,130774B1-DB21-4EF3-98C8-C104BCD6ED6D,2005-09-01
3,4,False,Ms.,Janet,M.,Gates,None,Modular Cycle Systems,adventure-works\jillian0,janet1@adventure-works.com,710-555-0173,ElzTpSNbUW1Ut+L5cWlfR7MF6nBZia8WpmGaQPjLOJA=,nm7D5e4=,FF862851-1DAA-4044-BE7C-3E85583C054D,2006-07-01
4,5,False,Mr.,Lucy,None,Harrington,None,Metropolitan Sports Supply,adventure-works\shu0,lucy0@adventure-works.com,828-555-0186,KJqV15wsX3PG8TS5GSddp6LFFVdd3CoRftZM/tP0+R4=,cNFKU4w=,83905BDC-6F5E-4F71-B162-C98DA069F38A,2006-09-01


In [17]:
datos.to_csv(
    'sales.csv',    # file name
    index=False,    # don’t write row index
    encoding='utf-8'  # use UTF-8 encoding
)


In [9]:
pip install pandas scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from datetime import datetime

# 1. Load data, parse ModifiedDate as datetime
df = datos

# 2. Convert dates to ordinal (integer) for regression
df["target"] = df["ModifiedDate"].apply(lambda dt: dt.toordinal())

# 3. Prepare features:
#    - Drop columns we won’t use (identifier, GUID, original date, password hashes)
#    - One-hot encode everything else with pandas
X = pd.get_dummies(
    df.drop(columns=[
        "CustomerID", "rowguid", "ModifiedDate",
        "PasswordHash", "PasswordSalt", "target"
    ]),
    drop_first=True
)
y = df["target"]

# 4. Split into train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 5. Fit a simple linear model
model = LinearRegression()
model.fit(X_train, y_train)

# 6. Make predictions
y_pred = model.predict(X_test)

# 7. Show first 5 predictions vs. true dates
for pred_ord, true_ord in zip(y_pred[:10], y_test.iloc[:10]):
    pred_date = datetime.fromordinal(int(pred_ord)).date()
    true_date = datetime.fromordinal(int(true_ord)).date()
    print(f"Predicted: {pred_date},  True: {true_date}")

Predicted: 2007-08-01,  True: 2007-08-01
Predicted: 2005-10-31,  True: 2005-11-01
Predicted: 2006-07-01,  True: 2006-07-01
Predicted: 2005-07-01,  True: 2005-07-01
Predicted: 2010-07-13,  True: 2006-08-01
Predicted: 2007-07-31,  True: 2007-08-01
Predicted: 2007-08-31,  True: 2007-09-01
Predicted: 2011-08-26,  True: 2007-07-01
Predicted: 2002-07-15,  True: 2005-09-01
Predicted: 2005-09-01,  True: 2005-09-01
